In [ ]:
!pip install requests beautifulsoup4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install selenium

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import random

# 配置Chrome选项
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 初始化WebDriver
driver = webdriver.Chrome(options=chrome_options)
# 使用Selenium获取逐字稿链接
def get_transcript_links_selenium(driver, url, max_transcripts = max_transcripts_to_fetch):
    transcript_links = []
    page_number = 1
    url_usage_count = 0
    try:
        while len(transcript_links) < max_transcripts:
            driver.get(url)
            print(url)
            time.sleep(random.uniform(5, 25))  # 等待页面初次加载
            # 增加隐式等待时间
            driver.implicitly_wait(30)
            # 解析新加载的内容
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_links = soup.find_all('a', class_='text-share-text visited:text-share-text hover:text-share-text focus:text-share-text', href=True)
            print(len(new_links))

            for link in new_links:
                href = link['href']
                if href.startswith("/article/") and href not in transcript_links:
                    full_url = f'https://seekingalpha.com/{href}'
                    transcript_links.append(full_url)
                    if len(transcript_links) >= max_transcripts:
                        break
            if len(new_links)==0:
                url_usage_count = url_usage_count + 1
                url = url
                if url_usage_count%2 == 0:
                  page_number = page_number + 1
                  print(page_number)
                  url = f'https://seekingalpha.com/earnings/earnings-call-transcripts?page={page_number}'
            if len(new_links)==40:
                page_number = page_number + 1
                print(page_number)
                url = f'https://seekingalpha.com/earnings/earnings-call-transcripts?page={page_number}'
        return transcript_links
    finally:
        print("over")
# 指定要抓取的网页
start_url = 'https://seekingalpha.com/earnings/earnings-call-transcripts?page=1'
max_transcripts_to_fetch = 1000  # 您想要抓取的逐字稿数量

# 获取逐字稿链接
transcripts = get_transcript_links_selenium(driver, start_url, max_transcripts_to_fetch)

# 关闭WebDriver
driver.quit()

# 打印获取到的逐字稿链接
for transcript in transcripts:
    print(transcript)